# Task 2 - User Engagement Analysis

calculate r engagement metric

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('your_dataset.csv')

# Calculate engagement metrics per user and application
engagement_metrics = df.groupby(['user_id', 'application']).agg(
    total_session_duration=('session_duration', 'sum'),
    number_of_sessions=('session_id', 'count'),
    total_download=('download_data', 'sum'),
    total_upload=('upload_data', 'sum'),
).reset_index()

# Define engagement score (example: weighted sum)
engagement_metrics['engagement_score'] = (
    engagement_metrics['total_session_duration'] * 0.5 +
    engagement_metrics['number_of_sessions'] * 0.3 +
    (engagement_metrics['total_download'] + engagement_metrics['total_upload']) * 0.2
)
print(engagement_metrics)

analyse engagement metric

In [ ]:
# Calculate average engagement score per application
app_engagement = engagement_metrics.groupby('application').agg(
    average_engagement_score=('engagement_score', 'mean'),
    total_users=('user_id', 'nunique')
).reset_index().sort_values(by='average_engagement_score', ascending=False)

print(app_engagement)

visualize engagement levels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Bar plot of average engagement scores
plt.figure(figsize=(10, 6))
sns.barplot(data=app_engagement, x='average_engagement_score', y='application')
plt.title('Average Engagement Scores by Application')
plt.xlabel('Average Engagement Score')
plt.ylabel('Application')
plt.show()

or

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the metrics
plt.figure(figsize=(15, 5))

# Sessions Frequency
plt.subplot(1, 3, 1)
sns.histplot(engagement_metrics['sessions_frequency'], bins=30, kde=True)
plt.title('Sessions Frequency Distribution')
plt.xlabel('Sessions Frequency')
plt.ylabel('Count')

# Total Session Duration
plt.subplot(1, 3, 2)
sns.histplot(engagement_metrics['total_session_duration'], bins=30, kde=True)
plt.title('Total Session Duration Distribution')
plt.xlabel('Total Session Duration (seconds)')
plt.ylabel('Count')

# Total Traffic
plt.subplot(1, 3, 3)
sns.histplot(engagement_metrics['total_traffic'], bins=30, kde=True)
plt.title('Total Traffic Distribution')
plt.xlabel('Total Traffic (bytes)')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

calculate engagement metrics

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('your_dataset.csv')

# Calculate engagement metrics per user and application
engagement_metrics = df.groupby(['user_id', 'application']).agg(
    sessions_frequency=('session_id', 'count'),  # Total number of sessions
    total_session_duration=('session_duration', 'sum'),  # Duration of all sessions
    total_traffic=('download_data', 'sum') + ('upload_data', 'sum')  # Total traffic in bytes
).reset_index()

# Rename columns for clarity
engagement_metrics.rename(columns={
    'sessions_frequency': 'sessions_frequency',
    'total_session_duration': 'total_session_duration',
    'total_traffic': 'total_traffic'
}, inplace=True)

# Display the engagement metrics
print(engagement_metrics)

analyse engagement metrics

In [ ]:
# Summary statistics
summary_statistics = engagement_metrics[['sessions_frequency', 'total_session_duration', 'total_traffic']].describe()
print(summary_statistics)

Aggregate the above metrics per customer id (MSISDN)

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('your_dataset.csv')

# Aggregate metrics per customer ID (MSISDN)
engagement_metrics = df.groupby('MSISDN').agg(
    sessions_frequency=('session_id', 'count'),
    total_session_duration=('session_duration', 'sum'),
    total_traffic=('download_data', 'sum') + ('upload_data', 'sum')
).reset_index()

# Report the top 10 customers per engagement metric
top_customers_frequency = engagement_metrics.nlargest(10, 'sessions_frequency')
top_customers_duration = engagement_metrics.nlargest(10, 'total_session_duration')
top_customers_traffic = engagement_metrics.nlargest(10, 'total_traffic')

print("Top 10 Customers by Session Frequency:")
print(top_customers_frequency)

print("Top 10 Customers by Total Session Duration:")
print(top_customers_duration)

print("Top 10 Customers by Total Traffic:")
print(top_customers_traffic)

Normalize each engagement metric and run a k-means (k=3) to classify customers 
in three groups of engagement.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Normalize the engagement metrics
scaler = StandardScaler()
normalized_metrics = scaler.fit_transform(engagement_metrics[['sessions_frequency', 'total_session_duration', 'total_traffic']])

# Run K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
engagement_clusters = kmeans.fit_predict(normalized_metrics)

# Add cluster labels to the original DataFrame
engagement_metrics['cluster'] = engagement_clusters

Compute total non-normalized metrics for each cluster. Interpret your results visually with accompanying text explaining your 
findings.

In [ ]:
# Compute min, max, average & total non-normalized metrics for each cluster
cluster_summary = engagement_metrics.groupby('cluster').agg(
    min_sessions_frequency=('sessions_frequency', 'min'),
    max_sessions_frequency=('sessions_frequency', 'max'),
    avg_sessions_frequency=('sessions_frequency', 'mean'),
    total_sessions_frequency=('sessions_frequency', 'sum'),
    min_total_session_duration=('total_session_duration', 'min'),
    max_total_session_duration=('total_session_duration', 'max'),
    avg_total_session_duration=('total_session_duration', 'mean'),
    total_total_session_duration=('total_session_duration', 'sum'),
    min_total_traffic=('total_traffic', 'min'),
    max_total_traffic=('total_traffic', 'max'),
    avg_total_traffic=('total_traffic', 'mean'),
    total_total_traffic=('total_traffic', 'sum')
).reset_index()

print(cluster_summary)

Aggregate user total traffic per application

In [ ]:
# Aggregate user total traffic per application
app_traffic = df.groupby('application').agg(
    total_traffic=('download_data', 'sum') + ('upload_data', 'sum')
).reset_index()

# Derive top 10 most engaged users per application
top_users_per_app = df.groupby(['application', 'MSISDN']).agg(
    total_traffic=('download_data', 'sum') + ('upload_data', 'sum')
).reset_index()

top_users_per_app = top_users_per_app.sort_values(by=['application', 'total_traffic'], ascending=False).groupby('application').head(10)

Plot the top 3 most used applications

In [ ]:
# Plotting the top 3 applications
top_apps = app_traffic.nlargest(3, 'total_traffic')

plt.figure(figsize=(8, 5))
sns.barplot(data=top_apps, x='total_traffic', y='application')
plt.title('Top 3 Most Used Applications by Total Traffic')
plt.xlabel('Total Traffic (bytes)')
plt.ylabel('Application')
plt.show()

optimize value of k using elbow method

In [ ]:
# Elbow method for optimal k
inertia = []
k_range = range(1, 10)  # Test for k from 1 to 9
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(normalized_metrics)
    inertia.append(kmeans.inertia_)

# Plot elbow method results
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_range)
plt.show()